In [ ]:
import os, random
import numpy as np
import pandas as pd
from PIL import Image

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader

from tqdm import tqdm
from sklearn.metrics import classification_report

import timm
from einops import rearrange

import cv2
import matplotlib.pyplot as plt

In [2]:
import pandas as pd
import zipfile
import os

# Direct download link
url = "https://www.dropbox.com/scl/fi/gp34duk5wdqea9fceys6v/scenario17.zip?rlkey=5758v2ne70qyddg7c8505ufv5&dl=1"

# Download the zip file
!wget -O scenario17.zip "$url"

# Unzip the contents
with zipfile.ZipFile("scenario17.zip", "r") as zip_ref:
    zip_ref.extractall("scenario17")

# List extracted files
os.listdir("scenario17")

data = pd.read_csv("/content/scenario17/scenario17.csv")
print(data.head())

--2026-02-05 05:21:46--  https://www.dropbox.com/scl/fi/gp34duk5wdqea9fceys6v/scenario17.zip?rlkey=5758v2ne70qyddg7c8505ufv5&dl=1
Resolving www.dropbox.com (www.dropbox.com)... 162.125.5.18, 2620:100:601d:18::a27d:512
Connecting to www.dropbox.com (www.dropbox.com)|162.125.5.18|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://uca714b1b029360628f417da3472.dl.dropboxusercontent.com/cd/0/inline/C6Rx3tA5we7M8uhkpFJT4la6ZnEcePjtPSVdyYyX-uJomkfPGNCU2hT1UDv373T2QnbuMhYWLjksHChOdJ5JAMqBph7EtdpQvhYMEXbr8xnogtiq9VEkCz-TBAAk4TaoLvTHiOwo1tyF649MqgxufSUZ/file?dl=1# [following]
--2026-02-05 05:21:47--  https://uca714b1b029360628f417da3472.dl.dropboxusercontent.com/cd/0/inline/C6Rx3tA5we7M8uhkpFJT4la6ZnEcePjtPSVdyYyX-uJomkfPGNCU2hT1UDv373T2QnbuMhYWLjksHChOdJ5JAMqBph7EtdpQvhYMEXbr8xnogtiq9VEkCz-TBAAk4TaoLvTHiOwo1tyF649MqgxufSUZ/file?dl=1
Resolving uca714b1b029360628f417da3472.dl.dropboxusercontent.com (uca714b1b029360628f417da3472.dl.dropboxusercontent.com)... 162

In [6]:
import os, random
import numpy as np
import pandas as pd
from PIL import Image

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader

from tqdm import tqdm
from sklearn.metrics import classification_report

import timm
from einops import rearrange

import cv2
import matplotlib.pyplot as plt

# ============================================================
# FINAL PIPELINE
# FAST + MULTI-TASK + XAI
# ViT + Signal Transformer + Attention
# Horizons: 1,5,10
# ============================================================




# ============================================================
# CONFIG
# ============================================================

BASE = "/content/scenario17"
CACHE = "cache_multi_fast"

BATCH = 8
EPOCHS = 5
LR = 1e-4
WORKERS = 2

SEQ = 8
HORIZONS = [1,5,10]

DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

os.makedirs(CACHE, exist_ok=True)


# ============================================================
# 1. LOAD DATA
# ============================================================

print("Loading CSV...")

df = pd.read_csv("/content/scenario17/scenario17.csv")


def fix(p):
    return os.path.join(BASE, p.replace("./", ""))


for c in ["unit1_rgb","unit1_pwr_60ghz","unit1_blockage"]:
    df[c] = df[c].apply(fix)


def read_label(p):
    with open(p) as f:
        return int(float(f.readline()))


df["label"] = df["unit1_blockage"].apply(read_label)

print(df["label"].value_counts())


# ============================================================
# 2. WINDOW BUILDER (MULTI-HORIZON)
# ============================================================

def build_windows(df):

    windows = []

    if "seq_index" in df.columns:
        groups = df.groupby("seq_index")
    else:
        groups = [(0,df)]

    for _,g in groups:

        g = g.reset_index(drop=True)

        i = 0

        while i+SEQ+max(HORIZONS) < len(g):

            win = g.iloc[i:i+SEQ]

            labels = [
                g.iloc[i+SEQ-1+h]["label"]
                for h in HORIZONS
            ]

            windows.append((win,labels))

            i+=1

    print("Total windows:",len(windows))

    return windows


windows = build_windows(df)


# ============================================================
# 3. CACHED DATASET
# ============================================================

def load_signal(p,L=128):

    v = np.loadtxt(p)

    v = np.clip(v,-90,0)

    v = (v-v.mean())/(v.std()+1e-8)

    v = np.pad(v,(0,max(0,L-len(v))))[:L]

    return v.astype(np.float32)


class CachedDataset(Dataset):

    def __init__(self,windows,train=True):

        random.shuffle(windows)

        s = int(0.8*len(windows))

        if train:
            self.data = windows[:s]
            self.split = "train"
        else:
            self.data = windows[s:]
            self.split = "test"

        self.cache = os.path.join(CACHE,self.split)

        os.makedirs(self.cache,exist_ok=True)

    def __len__(self):
        return len(self.data)

    def _path(self,i):
        return os.path.join(self.cache,f"{i}.pt")

    def __getitem__(self,i):

        p = self._path(i)

        if os.path.exists(p):
            return torch.load(p)

        win,lab = self.data[i]

        imgs,sigs=[],[]

        for _,r in win.iterrows():

            # Image
            img = Image.open(r["unit1_rgb"]) \
                      .resize((224,224)) \
                      .convert("RGB")

            img = np.array(img)/255.0
            imgs.append(img.transpose(2,0,1))

            # Signal
            sigs.append(load_signal(r["unit1_pwr_60ghz"]))

        sample = (
            torch.tensor(np.array(imgs),dtype=torch.float32),
            torch.tensor(np.array(sigs),dtype=torch.float32),
            torch.tensor(lab,dtype=torch.long)
        )

        torch.save(sample,p)

        return sample


train_ds = CachedDataset(windows,True)
test_ds  = CachedDataset(windows,False)


def make_loader(ds,shuffle):

    return DataLoader(
        ds,
        batch_size=BATCH,
        shuffle=shuffle,
        num_workers=WORKERS,
        pin_memory=True,
        persistent_workers=True
    )


train_dl = make_loader(train_ds,True)
test_dl  = make_loader(test_ds,False)


# ============================================================
# 4. MODELS (FAST)
# ============================================================

# ---------- ViT (Frozen) ----------

class ViTEncoder(nn.Module):

    def __init__(self):

        super().__init__()

        self.vit = timm.create_model(
            "vit_small_patch16_224",
            pretrained=True,
            num_classes=0
        )

        for p in self.vit.parameters():
            p.requires_grad = False

    def forward(self,x):

        B,S,C,H,W = x.shape

        x = rearrange(x,"b s c h w -> (b s) c h w")

        f = self.vit(x)

        return rearrange(f,"(b s) d -> b s d",s=S)


# ---------- Signal Transformer ----------

class SignalEncoder(nn.Module):

    def __init__(self,dim=128):

        super().__init__()

        layer = nn.TransformerEncoderLayer(
            dim,4,batch_first=True
        )

        self.enc = nn.TransformerEncoder(layer,2)

    def forward(self,x):

        return self.enc(x)


# ---------- Fusion ----------

class Fusion(nn.Module):

    def __init__(self):

        super().__init__()

        self.vp = nn.Linear(384,256)
        self.sp = nn.Linear(128,256)

        self.att = nn.MultiheadAttention(
            256,4,batch_first=True
        )

    def forward(self,v,s):

        v = self.vp(v.mean(1)).unsqueeze(1)
        s = self.sp(s.mean(1)).unsqueeze(1)

        x = torch.cat([v,s],1)

        out,att = self.att(x,x,x)

        return out.mean(1),att


# ---------- Multi-Task Net ----------

class MultiTaskNet(nn.Module):

    def __init__(self):

        super().__init__()

        self.vit = ViTEncoder()
        self.sig = SignalEncoder()
        self.fus = Fusion()

        self.h1  = nn.Linear(256,2)
        self.h5  = nn.Linear(256,2)
        self.h10 = nn.Linear(256,2)

    def forward(self,img,sig):

        v = self.vit(img)
        s = self.sig(sig)

        f,att = self.fus(v,s)

        return (
            self.h1(f),
            self.h5(f),
            self.h10(f),
            att
        )


# ============================================================
# 5. TRAINING (AMP)
# ============================================================

model = MultiTaskNet().to(DEVICE)

opt = torch.optim.AdamW(
    list(model.sig.parameters())+
    list(model.fus.parameters())+
    list(model.h1.parameters())+
    list(model.h5.parameters())+
    list(model.h10.parameters()),
    LR
)

loss_fn = nn.CrossEntropyLoss()

scaler = torch.cuda.amp.GradScaler()


print("Training (FAST MODE)...")

for e in range(EPOCHS):

    model.train()

    total = 0

    for img,sig,y in tqdm(train_dl):

        img = img.to(DEVICE,non_blocking=True)
        sig = sig.to(DEVICE,non_blocking=True)
        y   = y.to(DEVICE,non_blocking=True)

        opt.zero_grad()

        with torch.cuda.amp.autocast():

            o1,o5,o10,_ = model(img,sig)

            loss = (
                loss_fn(o1,y[:,0]) +
                loss_fn(o5,y[:,1]) +
                loss_fn(o10,y[:,2])
            )

        scaler.scale(loss).backward()
        scaler.step(opt)
        scaler.update()

        total += loss.item()

    print(f"Epoch {e+1} | Loss: {total/len(train_dl):.4f}")


# ============================================================
# 6. EVALUATION
# ============================================================

print("Evaluating...")

model.eval()

YT = {1:[],5:[],10:[]}
YP = {1:[],5:[],10:[]}

with torch.no_grad():

    for img,sig,y in test_dl:

        img = img.to(DEVICE)
        sig = sig.to(DEVICE)

        o1,o5,o10,_ = model(img,sig)

        YT[1]+=y[:,0].tolist()
        YT[5]+=y[:,1].tolist()
        YT[10]+=y[:,2].tolist()

        YP[1]+=o1.argmax(1).cpu().tolist()
        YP[5]+=o5.argmax(1).cpu().tolist()
        YP[10]+=o10.argmax(1).cpu().tolist()


for h in [1,5,10]:

    print(f"\n--- Horizon {h} ---")

    print(classification_report(YT[h],YP[h]))


# ============================================================
# 7. GRAD-CAM (XAI)
# ============================================================

class GradCAM:

    def __init__(self,model):

        self.model = model
        self.grad = None
        self.act  = None

        self._register()

    def _register(self):

        def f_hook(m,i,o):
            self.act = o

        def b_hook(m,gi,go):
            self.grad = go[0]

        layer = self.model.vit.vit.blocks[-1]

        layer.register_forward_hook(f_hook)
        layer.register_backward_hook(b_hook)

    def generate(self,img,sig):

        self.model.eval()

        o1,_,_,_ = self.model(img,sig)

        score = o1[:,1].sum()

        self.model.zero_grad()
        score.backward()

        g = self.grad.mean(dim=(2,3),keepdim=True)

        cam = (g*self.act).sum(1)

        cam = torch.relu(cam)
        cam = cam/cam.max()

        return cam


# ============================================================
# 8. XAI DEMO
# ============================================================

print("Generating XAI...")

cam = GradCAM(model)

img,sig,y = test_ds[0]

img = img.unsqueeze(0).to(DEVICE)
sig = sig.unsqueeze(0).to(DEVICE)

heat = cam.generate(img,sig)[0].cpu().numpy()

orig = img[0,0].permute(1,2,0).cpu().numpy()

heat = cv2.resize(heat,(128,128))
heat = cv2.applyColorMap(
    np.uint8(255*heat),
    cv2.COLORMAP_JET
)

out = heat*0.4 + orig*255

plt.imshow(out.astype(np.uint8))
plt.axis("off")
plt.title("Explainable AI (Grad-CAM)")
plt.show()

Loading CSV...
label
0    68551
1     1449
Name: count, dtype: int64
Total windows: 69928


/tmp/ipython-input-1894613532.py:339: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = torch.cuda.amp.GradScaler()
/usr/local/lib/python3.12/dist-packages/torch/cuda/amp/grad_scaler.py:31: UserWarning: torch.cuda.amp.GradScaler is enabled, but CUDA is not available.  Disabling.
  super().__init__(


Training (FAST MODE)...


  0%|          | 0/6993 [00:00<?, ?it/s]/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:668: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)
/tmp/ipython-input-1894613532.py:183: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at /pytorch/torch/csrc/utils/tensor_new.cpp:253.)
  torch.tensor(imgs,dtype=torch.float32),
/tmp/ipython-input-1894613532.py:183: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at /pytorch/torch/csrc/utils/tensor_new.cpp:253.)
  torch.tensor(imgs,dtype=torch.float32),
/tmp/ipython-input-1894613532.py:358: FutureWarnin

RuntimeError: Caught RuntimeError in DataLoader worker process 0.
Original Traceback (most recent call last):
  File "/usr/local/lib/python3.12/dist-packages/torch/utils/data/_utils/worker.py", line 349, in _worker_loop
    data = fetcher.fetch(index)  # type: ignore[possibly-undefined]
           ^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/torch/utils/data/_utils/fetch.py", line 55, in fetch
    return self.collate_fn(data)
           ^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/torch/utils/data/_utils/collate.py", line 398, in default_collate
    return collate(batch, collate_fn_map=default_collate_fn_map)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/torch/utils/data/_utils/collate.py", line 212, in collate
    collate(samples, collate_fn_map=collate_fn_map)
  File "/usr/local/lib/python3.12/dist-packages/torch/utils/data/_utils/collate.py", line 155, in collate
    r